# **Credit Card Churn - PredictiveModelling**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Objective**

To build a predictive model that helps a bank identify customers at risk of churn, enabling the business to take proactive action in retaining valuable clients. This notebook focuses on the technical groundwork using Python — from data exploration to model development.

**Project Structure**

1.   Introduction
2.   Dataset Overview
3.   Data Cleaning & Exploratory Data Analysis(EDA)
4.   Feature Engineering
5.   Modelling
6.   Model Insights
7.   Conclusion



## 1. **Introduction**

**Business Problem**

The bank is facing a rising concern: an increasing number of customers are closing their credit card accounts. This churn trend is causing financial and strategic challenges, and leadership wants to understand which customers are likely to leave so they can intervene with tailored services or offers

**Project Objective**

This notebook (Part 1 of a 3-part analytical solution) tackles the problem using Python to:



*   Explore customer behavior and account data.
*   Engineer relevant features from raw variables.
*   Build predictive models that estimate churn risk.
*   Provide actionable insights that inform targeted customer retention strategies.



This groundwork leads into:



*   **Part 2**: Designing a Power BI dashboard for interactive visualizations.
*   **Part 3**: Conducting a business analysis to recommend retention strategies backed by data.




## 2. **Dataset Overview**

**Source**

This dataset is publicly available on [Kaggle: Credit Card Customers Dataset](https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers/data). It contains detailed information on 10,127 customers, including demographic attributes, account activity, and churn status.


**Key Features**

The dataset includes 18 variables that span across customer profiles and credit card usage patterns. Some notable features are:

- Customer_Age: Age of the customer
- Gender: Male or Female
- Dependent_count: Number of dependents
- Education_Level: Highest education attained
- Marital_Status: Marital status of the customer
- Income_Category: Estimated annual income
- Card_Category: Type of credit card held
- Months_on_book: Tenure with the bank
- Total_Trans_Ct: Number of transactions in the last 12 months
- Credit_Limit: Assigned credit card limit
- Attrition_Flag: Indicates whether the customer has churned


**Target Variable**

- Attrition_Flag: This binary feature identifies whether a customer is Existing or Attrited. It will serve as the target for churn prediction.


**Initial Observations**

- The dataset is clean and well-structured, with minimal missing values.
- Features are a mix of categorical and numerical types, suitable for both statistical analysis and machine learning.
- The churn rate appears imbalanced, which may require resampling techniques during modeling.


## 3. **Data Cleaning & Exploratory Data Analysis(EDA)**

## 4. **Feature Engineering**

## 5. **Modelling**

## 6. **Model Insights**

## 7. **Conclusion**